# Poverty Estimation

## Setup

In [1]:
SURVEY_NAME = "DHS"

In [2]:
## Libraries
import os, datetime
import numpy as np
import pandas as pd
#import geopandas as gpd
import json
#import rasterio
#from rasterio.plot import show
import matplotlib.pyplot as plt

from sklearn.preprocessing import KBinsDiscretizer, StandardScaler, normalize
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import Ridge

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (BaggingClassifier, AdaBoostClassifier,
                              AdaBoostRegressor,
                              GradientBoostingClassifier, RandomForestClassifier,
                              RandomForestRegressor,
                             BaggingRegressor, GradientBoostingRegressor)
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (accuracy_score, precision_score, 
                             recall_score, classification_report,
                            r2_score, mean_absolute_error, mean_squared_error)

from joblib import dump, load

import logging, os 

import grid_params as grids
import config as cf

In [3]:
OUT_DIR = os.path.join(cf.DROPBOX_DIRECTORY, 'data', SURVEY_NAME, 'FinalData', 'pov_estimation_results')

## Load Data

In [4]:
df = pd.read_csv(os.path.join(cf.DROPBOX_DIRECTORY, 'data', SURVEY_NAME, "FinalData", "Merged Datasets", "survey_alldata_clean.csv"))

/Users/robmarty/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (47,51,54,57,58,59,61) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()
#df = df[df.country_code != 'IA']
#df = df[df.country_code != 'TL']

,gadm_uid,uid,country_code,country_year,urban_rural,year,most_recent_survey,educ_years_hh_max,educ_years_hh_mean,water_time_to_get,...,osm_dist_track,osm_dist_path,osm_length_all_5000m,osm_N_segments_any_5000m,osm_dist_any,iso2,continent,un_region,un_subregion,continent_adj
0,1,AL201700000001,AL,AL_2017-18_DHS_09092021_165_82518,U,2017,True,15.291667,10.859722,1.250000,...,117.344652,54.112282,347461.146812,1104,22.714830,AL,Europe,Europe,Southern Europe,Eurasia
1,1,AL201700000002,AL,AL_2017-18_DHS_09092021_165_82518,U,2017,True,14.318182,12.049242,1.363636,...,15.875410,1062.050826,316750.367862,1021,15.875410,AL,Europe,Europe,Southern Europe,Eurasia
2,1,AL201700000003,AL,AL_2017-18_DHS_09092021_165_82518,U,2017,True,14.823529,10.858824,0.000000,...,146.705601,2.501082,314586.631326,1028,2.501082,AL,Europe,Europe,Southern Europe,Eurasia
3,1,AL201700000004,AL,AL_2017-18_DHS_09092021_165_82518,U,2017,True,13.875000,10.215575,0.416667,...,300.303562,652.519259,319270.394940,1029,30.081477,AL,Europe,Europe,Southern Europe,Eurasia
4,1,AL201700000005,AL,AL_2017-18_DHS_09092021_165_82518,U,2017,True,16.000000,14.084314,0.000000,...,108.278757,19.394781,347761.488908,1099,19.394781,AL,Europe,Europe,Southern Europe,Eurasia


## Functions

In [7]:
def trainmodel_valresult(df, country, est_type, target, parameters, feature_type):
    # DESCRIPTION: Estimate poverty for a specific country, feature set and 
    # poverty variable across a number of parameters
    # ARGS:
    # df: Dataframe
    # country: iso2
    # est_type: within_country or other_countries
    # target: dependent variable to predict
    # feature_type: features to use

    # Within country prediction; predict across folds within a country
    if est_type == 'within_country_cv':
        df_traintest = df[df.country_code == country]
        df_traintest.reset_index()
        
        df_traintest['fold'] = df_traintest['within_country_fold']
    else:
        df['fold'] = 'fold_1'
        
    # Train model on all countries except country i; predict on country i
    if est_type == 'global_country_pred':
        df_traintest = df
        df_traintest.reset_index()
    
    # Train model on all countries in a continent, except country i; predict
    # on country x
    if est_type == 'continent_africa_country_pred':
        df_traintest = df[(df.country_code == country) | (df.continent_adj == 'Africa')]
        df_traintest.reset_index()
        
    if est_type == 'continent_americas_country_pred':
        df_traintest = df[(df.country_code == country) | (df.continent_adj == 'Americas')]
        df_traintest.reset_index()
        
    if est_type == 'continent_eurasia_country_pred':
        df_traintest = df[(df.country_code == country) | (df.continent_adj == 'Eurasia')]
        df_traintest.reset_index()
        
    # Train on all countries in continent x and predict on countries in continent y
    if est_type == 'continent':
        df_traintest = df
        df['fold'] = df['continent_adj']
           
    # Initialize results and predictions dataframes
    results_df = pd.DataFrame() # results iterating over params
    
    y_df = df_traintest[['uid', 'country_code']]
    y_df = y_df.copy()
    y_df['y'] = df_traintest[target]
    #y_df['y'] = df_traintest.loc[:, (target)]
    y_df['target'] = target
    y_df['feature_type'] = feature_type
    y_df['est_type'] = est_type
    
    model_i = 0
    for i in parameters['regressors']:
        for j in parameters[i]:
            
            pred_dict = {
                'regressor': i,
                'params': j,
                'country': country,
                'est_type': est_type,
                'target': target,
                'model_i': model_i,
                'feature_type': feature_type
             }

            fold = 0
            y_df_parami = pd.DataFrame()
            for split_id in df_traintest.fold.unique():
                
                #### Separate into train and test
                if 'country_pred' in est_type:
                    df_train = df_traintest[df_traintest.country_code != country]
                    df_test = df_traintest[df_traintest.country_code == country]
                elif est_type == 'continent':
                    df_train = df_traintest[df_traintest['fold'] == split_id]
                    df_test = df_traintest[df_traintest['fold'] != split_id]
                else:
                    df_train = df_traintest[df_traintest['fold'] != split_id]
                    df_test = df_traintest[df_traintest['fold'] == split_id]

                #### Select features
                if (feature_type == "fb"):
                    x_train = df_train.filter(regex='^fb_', axis=1)
                    x_test = df_test.filter(regex='^fb_', axis=1)
                    
                if (feature_type == "osm"):
                    x_train = df_train.filter(regex='^osm_', axis=1)
                    x_test = df_test.filter(regex='^osm_', axis=1)
                    
                if (feature_type == "l8"):
                    x_train = df_train.filter(regex='^l8_', axis=1)
                    x_test = df_test.filter(regex='^l8_', axis=1)
                    
                if (feature_type == "l8_viirs"):
                    x_train = df_train.filter(regex='^l8_|^viirs_', axis=1)
                    x_test = df_test.filter(regex='^l8_|^viirs_', axis=1)
                    
                if (feature_type == "all"):
                    x_train = df_train.filter(regex ='^fb_|^l8_|^viirs_|^osm_|^worldpop_', axis=1)
                    x_test  = df_test.filter(regex  ='^fb_|^l8_|^viirs_|^osm_|^worldpop_', axis=1)
                    
                #### Prep Y Data
                y_train = df_train[target]
                y_test = df_test[target]

                #### Prep X Data: Normalize
                x_scaler = StandardScaler().fit(x_train)

                x_train = x_scaler.transform(x_train)
                x_test = x_scaler.transform(x_test)

                ### Initialize regressor, fit data, then append model to list
                regressor = eval(i)(**j)
                trained = regressor.fit(x_train, y_train)

                ### Results
                y_pred = trained.predict(x_test)
                
                pred_dict['r2_score_' + str(split_id)] = r2_score(y_test, y_pred)
                
                y_dict_foldi = {
                    'uid': df_test.uid
                 }
                y_dict_foldi['y_' + str(model_i)] = y_pred
                
                y_df_foldi = pd.DataFrame.from_dict(y_dict_foldi)
                y_df_parami = y_df_parami.append(y_df_foldi, ignore_index=True)

                fold += 1

            y_df = y_df.merge(y_df_parami, on = 'uid', how = 'right')
            y_df.to_csv(os.path.join(OUT_DIR, 'country_withincv',
                               'withincv_predicted_values_' + 
                               country + '_' +
                               est_type + '_' +
                               feature_type + '_' +
                               target + 
                               '_fbonly.csv'))
                
            results_df = results_df.append(pred_dict, ignore_index=True)
            results_df.to_csv(os.path.join(OUT_DIR, 'country_withincv',
                                           'withincv_results_' + 
                                           country + '_' +
                                           est_type + '_' +
                                           feature_type + '_' +
                                           target + 
                                           '_fbonly.csv'))
            model_i += 1
        
    return results_df, y_df

In [8]:
parameters = grids.GRID_REGRESS

In [9]:
results_all_df = pd.DataFrame()
y_all_df = pd.DataFrame()

# 'fb', 'osm', 'l8', 'l8_viirs', 'all'
for feature_type_i in ['osm']:
    for est_type_i in ['continent', 'continent_americas_country_pred', 'within_country_cv']:
        for cc_i in df.country_code.unique():
            for target in ['pca_allvars']: # 'wealth_index_score'
                      
                # For predicting country i using continent x, only use
                # other countries in the continent.
                if est_type_i == 'continent_africa_country_pred':
                    continent_i = df.continent_adj[df.country_code == cc_i].tolist()[0]
                    if continent_i != 'Africa':
                        continue
                        
                if est_type_i == 'continent_americas_country_pred':
                    continent_i = df.continent_adj[df.country_code == cc_i].tolist()[0]
                    if continent_i != 'Americas':
                        continue
                        
                if est_type_i == 'continent_eurasia_country_pred':
                    continent_i = df.continent_adj[df.country_code == cc_i].tolist()[0]
                    if continent_i != 'Eurasia':
                        continue
                        
                # Only need to run 'continent' once
                if est_type_i == 'continent':
                    if cc_i != 'AL':
                        continue
                
                # Print where at
                print(feature_type_i + ' // ' + est_type_i + ' // ' + cc_i + ' // ' + target)

                # Run ML Model
                results_df_i, y_df_i = trainmodel_valresult(df, cc_i, est_type_i, target, parameters, feature_type_i)

                # Grab results
                results_all_df = results_all_df.append(results_df_i, ignore_index=True)
                y_all_df = y_all_df.append(y_df_i, ignore_index=True)
                
                # Export CSVs; replace as process more models
                results_all_df.to_csv(os.path.join(OUT_DIR, 'results_fbonly_withincv.csv'))
                y_all_df.to_csv(os.path.join(OUT_DIR, 'ypred_fbonly_withincv.csv'))

osm // continent // AL // pca_allvars
b
b
b
b
b
b
b
b
b
b
b
b


KeyboardInterrupt: 

In [11]:
results_all_df.to_csv(os.path.join(OUT_DIR, 'results_fbonly_withincv.csv'))
y_all_df.to_csv(os.path.join(OUT_DIR, 'ypred_fbonly_withincv.csv'))

In [51]:
est_type = 'hello_country_pred'

In [53]:
'country_pred' in est_type

True